In [1]:
#written by Zilin Shen and Daniel de Mello
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import time
import numpy as np
from packet_capturer_daniel import PacketCapturer
from window_manager import WindowManager
from feature_extractor import FeatureExtractor
from numpy_generator import NumpyGenerator
from label_packet import generate_label_data, label_packets
from lstm import LSTM
from random_perturb import random_perturb
from pso import PSO

# the hyperparameters of window manager
window_size = 1
move_size = 0.1 #how long the movement of window
swnd = True
# the class of detector
# attack, infection,reconnaissance
kind = "attack"

# get the dataset from
# because of the wrong generate_dataset, training is test,test is training
test_dataset_name = "../set_1/test.pcap"
training_dataset_name = "../set_1/training.pcap"
test_label_name = "../set_1/test.label"
training_label_name = "../set_1/training.label"

training_packet_capturer = PacketCapturer(None,training_dataset_name)
training_packet_capturer.pcap2packets()
train_packets = training_packet_capturer.packets

test_packet_capturer = PacketCapturer(None,test_dataset_name)
test_packet_capturer.pcap2packets()
test_packets = test_packet_capturer.packets

print("length of train packets,",len(train_packets))
print("length of test packets,", len(test_packets))

# ----------------start labeling the train_packets and test_packets-------------------
train_label, train_ts = generate_label_data(training_label_name)
test_label, test_ts = generate_label_data(test_label_name)
print("the shape of train_label,",train_label.shape)
print("the shape of test_label,",test_label.shape)

# filter the packets with label, drop the packet without label
train_packets = label_packets(train_packets, train_label,train_ts)
print('length of training packets after labeling',len(train_packets))
test_packets = label_packets(test_packets, test_label,test_ts)
print("length of testing packets after labeling",len(test_packets))

# -----------start randomly perturbing packets--
#random_perturb(test_packets)

# ----------------start generating the windows with packets--------------
train_window_manager = WindowManager(train_packets,window_size,swnd,move_size)
#train_window_manager.add_packets(train_packets)
train_window_manager.process_packets()
train_windows = train_window_manager.windows
print("successfully get training windows, the number of windows is ", len(train_window_manager.windows))

test_window_manager = WindowManager(test_packets,window_size,swnd,move_size)
#test_window_manager.add_packets(test_packets)
test_window_manager.process_packets()
test_windows = test_window_manager.windows
print("successfully get testing window, the number of windows is ",len(test_window_manager.windows))


# ---------------start extracting features------------------- #
test_fe = FeatureExtractor(test_windows)
#test_fe.add_features()
test_fe.process_windows()
print(test_windows[0].stat,type(test_windows[0].stat))

train_fe = FeatureExtractor(train_windows)
#train_fe.add_features()
train_fe.process_windows()

os.makedirs('preprocessed', exist_ok=True)

for kind in ['attack', 'infection', 'reconnaissance']:

    #------------------ start processing the windows to numpy data---------------#
    test_data_generator = NumpyGenerator(test_windows,kind)
    test_data_generator.process_windows()
    print(test_data_generator.df,test_data_generator.dataset,test_data_generator.label)

    # the second parameter is the kind of detector
    train_data_generator = NumpyGenerator(train_windows,kind,True)
    train_data_generator.process_windows()
    print(train_data_generator.df,train_data_generator.dataset,train_data_generator.label)

    # -----------get the training and testing data of numpy format
    # the training data, use balancing dataset and label
    data = train_data_generator.dataset_smo
    label = train_data_generator.label_smo

    np.save('preprocessed/train_data_{}.npy'.format(kind), data)
    np.save('preprocessed/train_label_{}.npy'.format(kind), label)

    # the testing data
    test_data = test_data_generator.dataset
    test_label = test_data_generator.label

    np.save('preprocessed/test_data_{}.npy'.format(kind), test_data)
    np.save('preprocessed/test_label_{}.npy'.format(kind), test_label)


the count of label lines 20084
(20084,)


2023-04-24 21:36:32.555029: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oracle/instantclient_19_8:/opt/oracle/instantclient_19_8:/p/cuda-11.5
2023-04-24 21:36:32.555058: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol
ERROR:root:Not supported protocol


length of train packets, 20085
length of test packets, 17370
the count of label lines 20084
(20084,)
the count of label lines 17372
(17372,)
the shape of train_label, (20084,)
the shape of test_label, (17372,)
length of training packets after labeling 19999
length of testing packets after labeling 17351
successfully get training windows, the number of windows is  10016
successfully get testing window, the number of windows is  4233
{'b_iat_max': 0.969084, 'b_iat_mean': 0.52, 'b_iat_min': 0.0007390000000000001, 'b_iat_std': 0.29, 'b_iat_total': 57.63674400000001, 'b_pkt_len_max': 66, 'b_pkt_len_mean': 66.0, 'b_pkt_len_min': 66, 'b_pkt_len_std': 0.0, 'flow_ack': 299, 'flow_cwr': 0, 'flow_ece': 0, 'flow_fin': 0, 'bpkt_per_second': 110.0, 'f_i_max': 0.972174, 'f_i_mean': 0.53, 'f_i_std': 0.28, 'f_i_min': 0.00309, 'f_i_total': 157.77104000000003, 'f_pro': 6, 'f_psh': 57, 'f_rst': 0, 'f_syn': 0, 'f_urg': 0, 'fpps': 299.0, 'fo_i_max': 0.971804, 'fo_i_min': 0.053155, 'fo_i_mean': 0.053155, 'fo

In [3]:
kind = 'infection'
#------------------ start processing the windows to numpy data---------------#
test_data_generator = NumpyGenerator(test_windows,kind)
test_data_generator.process_windows()
print(test_data_generator.df,test_data_generator.dataset,test_data_generator.label)

# the second parameter is the kind of detector
train_data_generator = NumpyGenerator(train_windows,kind, False)
train_data_generator.process_windows()
print(train_data_generator.df,train_data_generator.dataset,train_data_generator.label)

# -----------get the training and testing data of numpy format
# the training data, use balancing dataset and label
data = train_data_generator.dataset
label = train_data_generator.label

np.save('preprocessed/train_data_{}.npy'.format('infection_unbalanced'), data)
np.save('preprocessed/train_label_{}.npy'.format('infection_unbalanced'), label)

# the testing data
test_data = test_data_generator.dataset
test_label = test_data_generator.label

np.save('preprocessed/test_data_{}.npy'.format('infection_unbalanced'), test_data)
np.save('preprocessed/test_label_{}.npy'.format('infection_unbalanced'), test_label)



    b_iat_max  b_iat_mean     b_iat_min  b_iat_std  b_iat_total  \
0    0.969084        0.52      0.000739       0.29    57.636744   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
..        ...         ...           ...        ...          ...   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   
0    0.022234        0.01      0.022234       0.01     0.022234   
0   -1.000000       -1.00  99999.000000      -1.00     0.000000   

    b_pkt_len_max  b_pkt_len_mean  b_pkt_len_min  b_pkt_len_std  flow_ack  \
0              66            66.0             66            0.0       299   
0              -1             0.0        

In [10]:
data_generators_train['attack'].label

array([0., 0., 0., ..., 1., 1., 1.])

In [9]:
data_generators_train['infection'].label

array([1., 0., 1., ..., 0., 0., 0.])